In [1]:
# Import Modules
%load_ext autoreload
%autoreload 2

from math import pi
import pandas as pd
import sys
from pathlib import Path
from dotenv import load_dotenv

import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import HoverTool

In [2]:
# Set paths

PROJECT_ROOT = !git rev-parse --show-toplevel
PROJECT_ROOT = Path(PROJECT_ROOT[0])
print(PROJECT_ROOT)
sys.path.append(str(PROJECT_ROOT))
sys.path.append("/workspaces/template.python.project/tests")

/workspaces/template.python.project


In [3]:
from mock_dataset import mock_dataset

In [4]:
specs = {"float": [200, 1, 0] \
                        ,"int": [200, 1, 0] \
                        ,"categorical": [200, 1, 0] \
                        ,"bool": [200, 1, 0] \
                        ,"str": [200, 1, 0] \
                        ,"datetime": [200, 1, 0] \
                        }
df = mock_dataset(specs=specs)


In [5]:
df

,float_0,int_0,categorical_0,bool_0,str_0,datetime_0
0,0.130796,23412.0,BBC,1.0,KFCOTJGDYEHYATHHFUZUSTPMFZUPTZSZGQJFVLFFKLZSLV...,2021-02-03 11:01:10.235332
1,0.639820,110782.0,BCB,0.0,SADNKBGGJJXLYOZZRIAWONSAZAAUIUSEMAAOUZUWBQUDTM...,2021-02-04 11:01:10.235332
2,0.629856,65765.0,BAD,1.0,CMVROQUFBUFIBHJNDMEZFFOGDFHCQGVQPEXVLKGIYCNFDJ...,2021-02-05 11:01:10.235332
3,0.525743,134642.0,BDA,1.0,RPJFRANBMEGAQDBCWYZLQOEUBPMZNCYJPMHTXQRUFKBXDK...,2021-02-06 11:01:10.235332
4,0.388053,121577.0,CBA,1.0,SJYJPADZHNCWSTYGOATMOYGXUSORBQJEEPXFKUKYNDLXXI...,2021-02-07 11:01:10.235332
...,...,...,...,...,...,...
195,0.645895,131861.0,BBA,1.0,TOMTWBIOGGHSORAOAKCBXVWPZIMHBDDVGOIADRCENYLHGC...,2021-08-17 11:01:10.235332
196,0.474710,115701.0,ACB,1.0,NNMHRFBZIAWYWENZDFLEZCRBAFXGFLJMUCTIXLIRTOMDHS...,2021-08-18 11:01:10.235332
197,0.217618,81451.0,CBD,0.0,ALTIGHKXWFRPWEJFGRMSOTCMXFJMPOQWBWFUFLOQGNUPSM...,2021-08-19 11:01:10.235332
198,0.052062,22646.0,CCB,0.0,PBJXFFNJQPCEIOLHSAKSHUZFCEYEYXVVQZWQWRLIOGVVON...,2021-08-20 11:01:10.235332


# Line Plot

In [51]:
output_notebook()

x = "datetime_0"
y = "float_0"

# create a plot and style its properties
line = figure(plot_width=1000, plot_height=400, title=f"{y} vs {x}")
line.yaxis.axis_label = y
line.xaxis.axis_label = x

# create line renderer
line.line(x=x, y=y, line_width=2, source=df)
line.circle(x=x, y=y, source=df)

line.grid.grid_line_alpha=0.3

# Format the tooltip
tooltips = [(y, f"@{y}"), (x, f"@{x}")]

# Configure a renderer to be used upon hover
hover_glyph = line.circle(x=x, y=y, source=df, size=15, alpha=0, hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
line.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

show(line)

Loading BokehJS ...

# Bar Plot

In [53]:
output_notebook()

x = "datetime_0"
y = "int_0"

bar = figure(plot_width=1000, plot_height=400, title=f"{y} vs {x}")

bar.vbar(x=x, top=y, source=df)

# Format the tooltip
tooltips = [(y, f"@{y}"), (x, f"@{x}")]

# Configure a renderer to be used upon hover
hover_glyph = bar.circle(x=x, y=y, source=df, size=15, alpha=0, hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
bar.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

bar.grid.grid_line_alpha=0.3

show(bar)

Loading BokehJS ...

# Combine Line and Bar Plots

In [56]:
output_notebook()

x = "datetime_0"
y_line = "float_0"
y_bar = "int_0"

# create a plot and style its properties
line = figure(plot_width=1000, plot_height=400, title=f"{y_line} vs {x}")
line.yaxis.axis_label = y_line

# create line renderer
line.line(x=x, y=y_line, line_width=2, source=df)
line.circle(x=x, y=y_line, source=df)

line.grid.visible = False
line.xaxis.visible = False

# Format the tooltip
tooltips = [(y_line, f"@{y_line}"), (x, f"@{x}")]

# Configure a renderer to be used upon hover
hover_glyph = line.circle(x=x, y=y_line, source=df, size=15, alpha=0, hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
line.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

bar = figure(plot_width=1000, plot_height=400, title=f"{y_bar} vs {x}", x_range=line.x_range)

bar.vbar(x=x, top=y, source=df)

# Format the tooltip
tooltips = [(y_bar, f"@{y_bar}"), (x, f"@{x}")]

# Configure a renderer to be used upon hover
hover_glyph = bar.circle(x=x, y=y_bar, source=df, size=15, alpha=0, hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
bar.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

# 
bar.yaxis.axis_label = y_bar
bar.xaxis.axis_label = x
bar.grid.visible = False


show(gridplot([[line],[bar]]))

Loading BokehJS ...